## Descripción del dataset: Pima Indians Diabetes

El **Pima Indians Diabetes Dataset** es un conjunto de datos clásico en Machine Learning y bioestadística, recopilado por el *National Institute of Diabetes and Digestive and Kidney Diseases*.  
Su propósito es **predecir la aparición de diabetes tipo 2** en mujeres de origen **pima** (una población indígena del sur de Arizona, EE.UU.), a partir de diversas variables clínicas y demográficas.

### Características principales:
- **Número de registros:** 392 (en esta versión limpia, el original tenía 768).  
- **Número de atributos (features):** 8 variables predictoras + 1 variable objetivo.  
- **Población:** Mujeres de al menos 21 años de edad de la etnia Pima.  
- **Tarea principal:** Clasificación binaria → determinar si una paciente tiene diabetes (`Outcome = 1`) o no (`Outcome = 0`).

### Variables:
1. **Pregnancies** → Número de embarazos.  
2. **Glucose** → Concentración de glucosa en plasma después de 2 horas en una prueba de tolerancia a la glucosa.  
3. **BloodPressure** → Presión arterial diastólica (mm Hg).  
4. **SkinThickness** → Espesor del pliegue cutáneo del tríceps (mm).  
5. **Insulin** → Nivel sérico de insulina (mu U/ml).  
6. **BMI** → Índice de masa corporal (peso en kg / altura² en m²).  
7. **DiabetesPedigreeFunction** → Probabilidad de diabetes basada en antecedentes familiares.  
8. **Age** → Edad en años.  
9. **Outcome** → Variable objetivo:  
   - `0` = No tiene diabetes  
   - `1` = Tiene diabetes  

### Relevancia:
Este dataset es ampliamente utilizado en cursos de **Inteligencia Artificial y Machine Learning** para enseñar:
- Procesamiento y limpieza de datos biomédicos.  
- Métodos de clasificación supervisada (KNN, regresión logística, Random Forest, SVM, redes neuronales, etc.).  
- Importancia de la normalización y estandarización en algoritmos basados en distancias.  

---


## Paso 1: Cargar la base de datos  
Cargamos el CSV en un `DataFrame` de `pandas`. Si tu archivo no se llama exactamente `cleaned_dataset.csv`, ajusta la ruta.

In [ ]:
import pandas as pd
from pathlib import Path

# Cargar el dataset limpio
DATA_PATH = Path('dataset') / 'cleaned_dataset.csv'
df = pd.read_csv(DATA_PATH)

print(f"Filas y columnas: {df.shape}")
df.head()

## Paso 2: Crear subconjuntos con 20 datos de **entrenamiento** y 20 de **testeo**
Seleccionaremos 40 muestras: 20 para entrenar y 20 para evaluar.

In [ ]:
from sklearn.model_selection import train_test_split

# Seleccionamos 40 muestras al azar para trabajar el ejemplo manual
subset = df.sample(n=40, random_state=42).reset_index(drop=True)

X_subset = subset.drop(columns=['Outcome'])
y_subset = subset['Outcome']

# 20 para entrenamiento, 20 para prueba
testable = 20
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(
    X_subset, y_subset, test_size=testable, random_state=42, stratify=y_subset
)

print(f"Entrenamiento: {X_train_small.shape}, Prueba: {X_test_small.shape}")
subset.head()

## Paso 3: Implementar la función de distancia euclidiana

**Instrucciones:**
- Escribe una función en Python que reciba dos vectores y calcule la distancia euclidiana entre ellos.
- Utiliza la siguiente fórmula matemática para la distancia euclidiana entre dos vectores $x$ y $y$ de $n$ dimensiones:

$$
d(x, y) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

- Prueba tu función con los siguientes dos ejemplos (cada vector corresponde a una fila del dataset):

| Embarazos | Glucosa | Presión Arterial | Grosor Piel | Insulina | IMC  | Función Hereditaria | Edad | Resultado |
|-----------|---------|------------------|-------------|----------|------|---------------------|------|-----------|
|     1     |   106   |        70        |      28     |   135    | 34.2 |        0.142        |  22  |     0     |
|     2     |   102   |        86        |      36     |   120    | 45.5 |        0.127        |  23  |     1     |

- Calcula la distancia euclidiana a mano y luego verifica que el resultado de tu función sea el mismo.
- La función debe imprimir el resultado del cálculo de la distancia euclidiana con los datos presentados.



In [ ]:
import numpy as np

def euclidean_distance(x, y):
    x = np.array(x, dtype=float)
    y = np.array(y, dtype=float)
    return np.sqrt(np.sum((x - y) ** 2))

# Prueba con los dos vectores indicados
vector_a = [1, 106, 70, 28, 135, 34.2, 0.142, 22, 0]
vector_b = [2, 102, 86, 36, 120, 45.5, 0.127, 23, 1]

distancia_demo = euclidean_distance(vector_a[:-1], vector_b[:-1])
print(f"Distancia euclidiana de ejemplo: {distancia_demo:.4f}")

## Paso 4: Implementar un clasificador KNN básico

**Instrucciones:**
- Escribe una función que, dado un punto de prueba, calcule la distancia a todos los puntos de entrenamiento utilizando tu función de distancia euclidiana.
- Selecciona los **k = 3** vecinos más cercanos y predice la clase mayoritaria entre ellos.
- Aplica tu función a las 10 muestras de prueba obtenidas previamente, utilizando las 10 muestras de entrenamiento como referencia.
- El script debe imprimir una tabla comparando el valor real de `Resultado` de cada muestra de prueba con el valor predicho por tu algoritmo.
- Considere que las tablas se pueden codificar con un formato similar al que se muestra en el siguiente código:

In [ ]:
from collections import Counter


def knn_predict(point, X_train, y_train, k=3):
    distancias = []
    for idx, fila in enumerate(X_train.values):
        dist = euclidean_distance(point, fila)
        distancias.append((dist, y_train.iloc[idx]))
    distancias.sort(key=lambda x: x[0])
    vecinos = [label for _, label in distancias[:k]]
    conteo = Counter(vecinos)
    # En caso de empate, se devuelve la clase con mayor frecuencia y menor etiqueta
    return max(conteo.items(), key=lambda x: (x[1], -x[0]))[0]


# Aplicamos el KNN manual a las 20 muestras de prueba
predicciones = [
    knn_predict(row, X_train_small, y_train_small, k=3)
    for row in X_test_small.values
]

resultados_manual = pd.DataFrame(
    {
        'Real': y_test_small.values,
        'Predicho': predicciones,
    }
)

accuracy_manual = (resultados_manual['Real'] == resultados_manual['Predicho']).mean()
print("Tabla de comparación (subset 20/20):")
print(resultados_manual.head(10))
print(f"Accuracy manual (subset): {accuracy_manual:.3f}")

## Paso 5: Usar toda la data con separación 80% entrenamiento / 20% testeo  

### Pasos:
1. Cargar todo el dataset.  
2. Separar variables (X) y etiquetas (y).  
3. Aplicar `train_test_split` con 80% para entrenamiento y 20% para testeo.  
4. Mantener la proporción de clases usando estratificación.  
5. Guardar los conjuntos de datos para usarlos en KNN.  

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Outcome'])
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

## Paso 6: Entrenar un KNN con los datos sin escalar (crudos) y calcular accuracy  

### Pasos:
1. Definir el valor de **k = 3** y el metodo **Euclidiano**.  
2. Entrenar el modelo KNN con los datos crudos (sin normalizar/estandarizar).  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** comparando predicciones con etiquetas reales.  
5. Guardar el resultado para la tabla comparativa.  


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_raw = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_raw.fit(X_train, y_train)

y_pred_raw = knn_raw.predict(X_test)
accuracy_raw = accuracy_score(y_test, y_pred_raw)

print(f"Accuracy datos crudos: {accuracy_raw:.3f}")

## Paso 7: Normalizar (Min-Max scaling) y entrenar KNN, luego calcular accuracy  

### Pasos:
1. Aplicar **normalización Min-Max** a los datos de entrenamiento y test.  
2. Entrenar el modelo KNN con los datos normalizados.  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** del modelo.  
5. Guardar el resultado para la tabla comparativa.  


In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
X_train_norm = minmax_scaler.fit_transform(X_train)
X_test_norm = minmax_scaler.transform(X_test)

knn_norm = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_norm.fit(X_train_norm, y_train)

y_pred_norm = knn_norm.predict(X_test_norm)
accuracy_norm = accuracy_score(y_test, y_pred_norm)

print(f"Accuracy datos normalizados (Min-Max): {accuracy_norm:.3f}")

## Paso 9: Estandarizar (Z-score) y entrenar KNN, luego calcular accuracy  

### Pasos:
1. Aplicar **estandarización Z-score** a los datos de entrenamiento y test.  
2. Entrenar el modelo KNN con los datos estandarizados.  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** del modelo.  
5. Guardar el resultado para la tabla comparativa.  


In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
X_train_std = std_scaler.fit_transform(X_train)
X_test_std = std_scaler.transform(X_test)

knn_std = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_std.fit(X_train_std, y_train)

y_pred_std = knn_std.predict(X_test_std)
accuracy_std = accuracy_score(y_test, y_pred_std)

print(f"Accuracy datos estandarizados (Z-score): {accuracy_std:.3f}")

## Paso 10/11: Tabla comparativa de accuracies  

### Pasos:
1. Reunir los resultados de accuracy de cada experimento:  
   - KNN sin escalar (80/20).  
   - KNN normalizado (80/20).  
   - KNN estandarizado (80/20).  
2. Crear una tabla con los resultados.  
3. Comparar el desempeño de cada método.  



In [ ]:
import pandas as pd

resultados = pd.DataFrame(
    {
        'Método': [
            'KNN crudo (80/20)',
            'KNN normalizado Min-Max (80/20)',
            'KNN estandarizado Z-score (80/20)',
        ],
        'Accuracy': [accuracy_raw, accuracy_norm, accuracy_std],
    }
)

resultados

---
## Preguntas de reflexión y aplicación



1. ¿Por qué es importante normalizar o estandarizar los datos antes de usar KNN?  



Normalizar o estandarizar garantiza que todas las variables aporten de manera equilibrada al cálculo de distancias. KNN es sensible a la escala: si una característica tiene valores numéricamente grandes (por ejemplo, glucosa) dominará la distancia sobre otras con rangos pequeños (como `Diabetes Pedigree Function`). Al llevar todas las dimensiones a un rango comparable, evitamos sesgos y mejoramos la estabilidad del modelo.

2. ¿Qué diferencias observaste en el accuracy entre los datos crudos, normalizados y estandarizados?  


El accuracy con datos crudos fue el más bajo, mientras que la normalización Min-Max y la estandarización Z-score mejoraron el desempeño (valores muy similares entre sí). Esto confirma que escalar los atributos ayuda a que la métrica euclidiana sea más representativa en KNN. En este caso la estandarización obtuvo una ligera ventaja.

3. Si aumentamos el valor de **k** (número de vecinos), ¿cómo crees que cambiaría el rendimiento del modelo?  


Al incrementar **k**, cada predicción se basa en un vecindario más amplio, reduciendo la varianza y haciendo el modelo menos sensible al ruido. Sin embargo, si k es demasiado grande puede promediar casos de clases distintas y perder capacidad discriminativa, disminuyendo el accuracy. Suelo elegir k pequeño e impar y luego ajustarlo con validación cruzada.

4. ¿Qué ventaja tiene implementar KNN manualmente antes de usar scikit-learn?  


Implementar KNN manualmente obliga a entender cómo se calculan las distancias, cómo se ordenan los vecinos y cómo se resuelven los empates. Esa intuición facilita interpretar el modelo, depurar posibles errores de preprocesamiento y luego aprovechar scikit-learn con mayor criterio (elegir métrica, k adecuado, ponderación, etc.).

5. ¿Qué limitaciones presenta KNN cuando se aplica a conjuntos de datos grandes o con muchas dimensiones?  

KNN requiere almacenar todo el set de entrenamiento y calcular distancias a todos los puntos en cada predicción, por lo que escala mal con muchos registros (costo O(n·d)). Además, en espacios de alta dimensión la distancia euclidiana pierde significado (mal de la dimensionalidad), lo que degrada el rendimiento y exige técnicas de reducción de dimensiones o indexación especializada.

---

## Rúbrica de evaluación: Práctica KNN

| Criterio | Descripción | Puntaje Máximo |
|----------|-------------|----------------|
| **1. Carga y exploración del dataset** | Carga correcta del archivo CSV, explicación de las variables y verificación de datos. | 15 pts |
| **2. Implementación manual de KNN** | Código propio para calcular distancias euclidianas, selección de vecinos y votación mayoritaria. | 20 pts |
| **3. Predicción individual (ejemplo aleatorio)** | Explicación clara del proceso paso a paso para un ejemplo de test. | 10 pts |
| **4. Uso de scikit-learn (KNN)** | Entrenamiento y evaluación con `train_test_split`, comparación con el método manual. | 15 pts |
| **5. Normalización y estandarización** | Aplicación correcta de Min-Max y Z-score, con cálculo de accuracy en cada caso. | 20 pts |
| **6. Tabla comparativa de accuracies** | Presentación clara de los resultados y comparación entre métodos. | 10 pts |
| **7. Reflexión y preguntas finales** | Respuestas a las preguntas de análisis planteadas (profundidad y claridad). | 10 pts |

**Total: 100 pts**
